# Preparing environment

In [1]:
import requests
import json
import pandas as pd
import psycopg2
import numpy as np
import logging
import sys


# Set up logging

In [2]:
class LevelRangeFilter(logging.Filter):
    """
    A logging filter that allows messages within a specified range of logging levels to pass through.

    Attributes:
        min_level (int): The minimum logging level that the filter allows.
        max_level (int): The maximum logging level that the filter allows.
    """
    def __init__(self, min_level, max_level):
        """
        Initializes the filter with minimum and maximum logging levels.

        Args:
            min_level (int): The minimum logging level.
            max_level (int): The maximum logging level.
        """
        super().__init__()
        self.min_level = min_level
        self.max_level = max_level

    def filter(self, record):
        """
        Determines if the specified record should be logged based on its level.

        Args:
            record (LogRecord): The log record to be checked.

        Returns:
            bool: True if the record's level is within the specified range, False otherwise.
        """
        # Filter records that are not in the specified level range
        return self.min_level <= record.levelno <= self.max_level


# Create a logger for this module
logger = logging.getLogger(__name__)

# Set the logging level for the logger
logger.setLevel(logging.DEBUG)
# Setting the logger level to DEBUG means that all log messages, regardless of their severity, will be processed and output by this logger.

# Create handlers
success_handler = logging.FileHandler('etl_success.log')
debug_handler = logging.FileHandler('etl_debug.log')
error_handler = logging.FileHandler('etl_errors.log')
console_handler = logging.StreamHandler()  # This handler will send logs to the console

# Create formatters and add them to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
success_handler.setFormatter(formatter)
debug_handler.setFormatter(formatter)
error_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Create filters to the handlers
success_filter = LevelRangeFilter(logging.INFO, logging.WARNING)
error_filter = LevelRangeFilter(logging.WARNING, logging.CRITICAL)
debug_filter = LevelRangeFilter(logging.DEBUG, logging.CRITICAL)
console_filter = LevelRangeFilter(logging.INFO, logging.CRITICAL)

# Add filters to the handlers
success_handler.addFilter(success_filter)
error_handler.addFilter(error_filter)
debug_handler.addFilter(debug_filter)
console_handler.addFilter(console_filter)

# check if the logger currently has any handlers attached to it.
# logger.handlers will return a list of handlers attached to logger
if not logger.handlers:  #If this list is empty  it means no handlers are currently attached to the logger.
    # Add handlers to the logger
    logger.addHandler(success_handler)
    logger.addHandler(debug_handler)
    logger.addHandler(error_handler)
    logger.addHandler(console_handler)
    

In [3]:
# Beginning of the ETL
logger.info('Logging is set up and ETL pipeline is going to be executed')

2024-07-09 00:01:21,148 - __main__ - INFO - Logging is set up and ETL pipeline is going to be executed


# Extract

**API DOC**
https://spoonacular.com/application/frontend/downloads/spoonacular-api-slides.pdf

**API Link** https://spoonacular.com/food-api

In [4]:
def extract_data (api_url, num_recipes=100):
    """
    Extract data from the given API URL.

    Parameters:
    api_url (str): The URL of the API to extract data from.
    num_recipes (int): The number of recipes to request from the API. Default is 100.

    Returns:
    dict or None: The data extracted from the API in JSON format, or None if there was an error.
    """
    # Parameters to be sent with the API request
    params = {'number': num_recipes}
    try:
        # Send a GET request to the API with the specified parameters
        response = requests.get(api_url, params=params)
        # Raise an exception for non-2xx status codes
        response.raise_for_status() 
        try:
            # Parse the response content as JSON
            data = response.json()
            return data
        except json.JSONDecodeError as e:
            # Log an error if JSON parsing fails
            logger.error(f"Error parsing JSON data: {e}")
            return None
    except requests.exceptions.RequestException as e:
        # Log an error if the API request fails
        logger.error(f"Error extracting data from API: {e}")
        return None

In [5]:
# Define the API key used to authenticate with the Spoonacular API
API_KEY = '9fe94823f081434989282d1622cfbc31'
# Construct the API URL by appending the API key to the base URL for random recipes
# The API_URL is composed of the base URL for the random recipes endpoint and the API key as a query parameter
API_URL = 'https://api.spoonacular.com/recipes/random?apiKey=' + API_KEY

# Call the extract_data function to fetch data from the API URL
# The extract_data function sends a GET request to the API_URL and returns the response data
data = extract_data(API_URL)

# Check if data was successfully extracted
if data:
    logger.debug(data) # Log the extracted data at the debug level
    logger.info("Data extracted successfully.") # Log a success message at the info level
else:
    # Log an error message if data extraction failed
    logger.error("An error occurred while fetching data.")

2024-07-09 00:01:22,808 - __main__ - INFO - Data extracted successfully.


# Transform

### Extract the counter for the id columns: to be able to append the rows of the database tables
here i will comment the code related to the shell scripting and initialize the counter manually to debug

In [6]:
# Check if the counter argument is provided
#if len(sys.argv) < 2:
    #print("Counter argument is missing.")
    #sys.exit(1)

# Get the counter from the command-line arguments
#counter = int(sys.argv[1])


counter=101

## Json to Pandas
you can import any json data into pandas, the keys will become columns names

In [7]:
# Check if the 'recipes' key exists in the data dictionary and contains a list
if 'recipes' in data and isinstance(data['recipes'], list):
    # Convert the list of recipe dictionaries into a pandas DataFrame
    df = pd.DataFrame(data['recipes'])
    
    # Display the first few rows of the DataFrame to verify its structure
    logger.debug(df.head())
    
    # Log the number of recipes extracted
    logger.info(f"Extracted {len(df)} recipes.")

    # Log detailed information about the DataFrame, such as column types and non-null counts
    logger.debug(df.info())

else:
    # Log an error if the 'recipes' key is missing or does not contain a list
    logger.error("'recipes' key is missing or does not contain a valid list in the data dictionary.")

2024-07-09 00:01:22,853 - __main__ - INFO - Extracted 100 recipes.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   vegetarian                100 non-null    bool   
 1   vegan                     100 non-null    bool   
 2   glutenFree                100 non-null    bool   
 3   dairyFree                 100 non-null    bool   
 4   veryHealthy               100 non-null    bool   
 5   cheap                     100 non-null    bool   
 6   veryPopular               100 non-null    bool   
 7   sustainable               100 non-null    bool   
 8   lowFodmap                 100 non-null    bool   
 9   weightWatcherSmartPoints  100 non-null    int64  
 10  gaps                      100 non-null    object 
 11  preparationMinutes        3 non-null      float64
 12  cookingMinutes            3 non-null      float64
 13  aggregateLikes            100 non-null    int64  
 14  healthScore

## Dealing with recipes

In [8]:
# Define the required columns for the dataframe
dfrecipes_COLUMNS = [
    'vegetarian', 'vegan', 'glutenFree', 'dairyFree', 'veryHealthy', 
    'cheap', 'veryPopular', 'sustainable', 'lowFodmap', 'pricePerServing', 
    'title', 'readyInMinutes', 'servings', 'sourceUrl', 'summary', 'license'
]
# Construct the dataframe
try:
    dfrecipes = pd.DataFrame(df, columns=dfrecipes_COLUMNS)
    logger.info(f"Constructed DataFrame with {len(dfrecipes)} recipes.")
    logger.debug(dfrecipes.head())
    logger.debug(dfrecipes.info())
except Exception as e:
    logger.error("Failed to construct DataFrame:", exc_info=True)


# Find duplicates based on the 'title' column
try:
    duplicates = dfrecipes[dfrecipes.duplicated(subset=['title'])]
    logger.debug(duplicates)
    if not duplicates.empty:
    # remove duplicates
        logger.warning(f"Found {len(duplicates)} duplicate recipes. Removing duplicates.")
        dfrecipes = dfrecipes.drop_duplicates(subset=['title'])
        dfrecipes.reset_index(drop=True, inplace=True)
        logger.info(f"DataFrame now contains {len(dfrecipes)} recipes after removing duplicates.")
    else:
        logger.info("No duplicates found")
except Exception as e:
    logger.error("Failed to handle duplicates", exc_info=True)


# Rename columns for clarity and consistency
COLUMNS_TO_RENAME_MAP={
    'vegetarian': 'is_vegetarian', 'vegan': 'is_vegan',
    'glutenFree': 'is_glutenFree', 'dairyFree': 'is_dairyFree',
    'veryHealthy': 'is_healthy', 'cheap': 'is_cheap',
    'veryPopular': 'is_Popular', 'sustainable': 'is_sustainable',
    'lowFodmap': 'is_lowFodmap', 'pricePerServing': 'price_per_serving', 
    'readyInMinutes': 'ready_min', 'sourceUrl': 'source_url', 
    'title': 'recipe_title'}
try:
    dfrecipes = dfrecipes.rename(columns=COLUMNS_TO_RENAME_MAP)
    logger.info("Renamed columns.")
except Exception as e:
    logger.error("Failed to rename columns", exc_info=True)

# Generate'id_recipe' column
try:
    dfrecipes['id_recipe'] = range(counter, counter + len(dfrecipes))
    logger.info("Added 'id_recipe'.")
    logger.debug(dfrecipes.head())
    logger.debug(dfrecipes.info())
except Exception as e:
    logger.error("Failed to add 'id_recipe' column", exc_info=True)





2024-07-09 00:01:22,880 - __main__ - INFO - Constructed DataFrame with 100 recipes.
2024-07-09 00:01:22,900 - __main__ - INFO - No duplicates found
2024-07-09 00:01:22,902 - __main__ - INFO - Renamed columns.
2024-07-09 00:01:22,904 - __main__ - INFO - Added 'id_recipe'.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   vegetarian       100 non-null    bool   
 1   vegan            100 non-null    bool   
 2   glutenFree       100 non-null    bool   
 3   dairyFree        100 non-null    bool   
 4   veryHealthy      100 non-null    bool   
 5   cheap            100 non-null    bool   
 6   veryPopular      100 non-null    bool   
 7   sustainable      100 non-null    bool   
 8   lowFodmap        100 non-null    bool   
 9   pricePerServing  100 non-null    float64
 10  title            100 non-null    object 
 11  readyInMinutes   100 non-null    int64  
 12  servings         100 non-null    int64  
 13  sourceUrl        100 non-null    object 
 14  summary          100 non-null    object 
 15  license          61 non-null     object 
dtypes: bool(9), float64(1), int64(2), object(4)
memory usage: 6.5+ 

## Dealing with ingredients

In [9]:
# Define the required columns for the ingredients DataFrame
dfIng_COLUMNS=['consistency', 'nameClean', 'aisle', 'name']  


# Define a function to a list of dictionaries
def extract_dicts(lst):
    """Extract dictionaries from a list of dictionaries and return a list"""
    return [d for d in lst if isinstance(d, dict)]
# Apply the function and sum the results
try:
    # 'all_ingredients' variable recives a list of all dictionaries in the extendedIngredients column
    # note that a dataframe row is a dictionary
    all_ingredients = df['extendedIngredients'].apply(extract_dicts).sum()
    logger.info(f"Extracted {len(all_ingredients)} ingredients.")
except Exception as e:
    logger.error("Failed to extract ingredient dictionaries", exc_info=True)
    all_ingredients = []

# Construct the ingredients DataFrame
try:
    dfIng = pd.DataFrame(all_ingredients, columns=dfIng_COLUMNS)
    logger.info(f"Constructed ingredients DataFrame with {len(dfIng)} ingredients.")
    logger.debug(dfIng.head())
    logger.debug(dfIng.info())
except Exception as e:
    logger.error("Failed to construct ingredients DataFrame", exc_info=True)
    dfIng = pd.DataFrame(columns=dfIng_COLUMNS)


try:
    # Find duplicates
    duplicates = dfIng[dfIng.duplicated()]
    logger.debug(duplicates)
    # Remove duplicates
    if not duplicates.empty:
        logger.warning(f"Found {len(duplicates)} duplicate ingredients. Removing duplicates.")
        dfIng = dfIng.drop_duplicates().reset_index(drop=True)
        logger.info(f"DataFrame now contains {len(dfIng)} ingredients after removing duplicates.")
    else:
        logger.info("No duplicates found.")
except Exception as e:
    logger.error("Failed to handle duplicates", exc_info=True)

# Handle missing values in the nameClean column
try:
    none_rows = dfIng[dfIng['nameClean'].isna()]
    logger.debug(none_rows)
    if not none_rows.empty:
        logger.warning(f"Found {len(none_rows)} null nameClean. Filling with name.")
        logger.debug(none_rows)
        dfIng['nameClean'] = dfIng['nameClean'].fillna(dfIng['name'])
        logger.info("Filled missing 'nameClean' values.")
    else:
        logger.info("No null 'nameClean' found.")
    logger.debug(dfIng.info())
except Exception as e:
    logger.error("Failed to handle null values in 'nameClean'", exc_info=True)

# Drop the 'name' column after filling 'nameClean'
try:
    dfIng.drop('name', axis=1, inplace=True)
    logger.info("Dropped 'name' column.")
    logger.debug(dfIng.info())
except Exception as e:
    logger.error("Failed to drop 'name' column", exc_info=True)

try:
    # Find duplicates
    duplicates = dfIng[dfIng.duplicated()]
    logger.debug(duplicates)
    # Remove duplicates
    if not duplicates.empty:
        logger.warning(f"Found {len(duplicates)} duplicate ingredients. Removing duplicates.")
        dfIng = dfIng.drop_duplicates().reset_index(drop=True)
        logger.info(f"DataFrame now contains {len(dfIng)} ingredients after removing duplicates.")
    else:
        logger.info("No duplicates found.")
except Exception as e:
    logger.error("Failed to handle duplicates", exc_info=True)

# Rename 'nameClean' to 'ing_name'
try:
    dfIng = dfIng.rename(columns={'nameClean': 'ing_name'})
    logger.info("Renamed 'nameClean' to 'ing_name'.")
    logger.debug(dfIng.info())
except Exception as e:
    logger.error("Failed to rename columns", exc_info=True)

# Generate 'id_ingredient' column
try:
    dfIng['id_ingredient'] = range(counter, counter +len(dfIng))
    logger.info("Added 'id_ingredient'.")
    logger.debug(dfIng.head())
    logger.debug(dfIng.info())
except Exception as e:
    logger.error("Failed to add 'id_ingredient' column", exc_info=True)


2024-07-09 00:01:22,947 - __main__ - INFO - Extracted 1090 ingredients.
2024-07-09 00:01:22,951 - __main__ - INFO - Constructed ingredients DataFrame with 1090 ingredients.
2024-07-09 00:01:22,970 - __main__ - WARNING - Found 554 duplicate ingredients. Removing duplicates.
2024-07-09 00:01:22,975 - __main__ - INFO - DataFrame now contains 536 ingredients after removing duplicates.
2024-07-09 00:01:22,979 - __main__ - WARNING - Found 4 null nameClean. Filling with name.
2024-07-09 00:01:22,983 - __main__ - INFO - Filled missing 'nameClean' values.
2024-07-09 00:01:22,990 - __main__ - INFO - Dropped 'name' column.
2024-07-09 00:01:22,999 - __main__ - WARNING - Found 132 duplicate ingredients. Removing duplicates.
2024-07-09 00:01:23,002 - __main__ - INFO - DataFrame now contains 404 ingredients after removing duplicates.
2024-07-09 00:01:23,004 - __main__ - INFO - Renamed 'nameClean' to 'ing_name'.
2024-07-09 00:01:23,010 - __main__ - INFO - Added 'id_ingredient'.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090 entries, 0 to 1089
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   consistency  1090 non-null   object
 1   nameClean    1082 non-null   object
 2   aisle        1089 non-null   object
 3   name         1090 non-null   object
dtypes: object(4)
memory usage: 34.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   consistency  536 non-null    object
 1   nameClean    536 non-null    object
 2   aisle        535 non-null    object
 3   name         536 non-null    object
dtypes: object(4)
memory usage: 16.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   consistency  536 non-null

## Dealing with measures (reference_ing)

In [10]:
# Extract the column that contains all the ingredients measures along with the recipes titles
try:
    dfrecipeIng = pd.DataFrame(df, columns=['extendedIngredients', 'title'])
    logger.info(f"Extracted ingredients and titles for {len(dfrecipeIng)} recipes.")
    logger.debug(dfrecipeIng.head())
    logger.debug(dfrecipeIng.info())
except Exception as e:
    logger.error("Failed to extract ingredients and titles", exc_info=True)

#----------------------------------------------------------------------
# Obtaining a new dataframe that contains measures of ingredients in each recipe

# Define the required columns for the measures DataFrame
dfmeasures_COLUMNS =['nameClean', 'name', 'measures', 'title']

# Initialize an empty list to collect rows of data
rows_data = []

# Iterate through each row of the original DataFrame
for i, row_series in dfrecipeIng.iterrows():
    try:
        # Access the recipe title for the current row 
        title = row_series['title']  # title is a string
        
        # Access the 'extendedIngredients' column for the current row
        recipe_ingredients_list = row_series['extendedIngredients']  # recipe_ingredients_list is a list
    
        # if recipe_ingredients_list is not an empty list skip the iteration (according to the Guard clauses principle of clean code)
        if not (recipe_ingredients_list and isinstance(recipe_ingredients_list, list)):
            continue
            
        # Iterate through each dictionary in the list
        for each_dict in recipe_ingredients_list:
            if each_dict and isinstance(each_dict, dict):  # Check if each_dict is non-empty and is a dictionary
                # Get measures dict 
                measures_dict = each_dict.get('measures', {})
                # Transform measures dict to a list
                measures_list = [measures_dict]       
            else:
                measures_list = None
            
            # Create a dictionary for the row data
            row_data = {
                'nameClean': each_dict.get('nameClean', None),
                'measures': measures_list,
                'name': each_dict.get('name'),
                'title': title
            }
            
            # Append the row data to the list
            rows_data.append(row_data)
    except Exception as e:
        logger.error(f"Error processing row {i} for recipe '{title}'", exc_info=True)

# Create the DataFrame from the list of row_data
try:
    dfmeasures = pd.DataFrame(rows_data, columns=dfmeasures_COLUMNS)
    logger.info(f"Constructed measures DataFrame with {len(dfmeasures)} rows.")
    logger.debug(dfmeasures.head())
    logger.debug(dfmeasures.info())
except Exception as e:
    logger.error("Failed to construct measures DataFrame", exc_info=True)

# Handle missing values in the nameClean column
try:
    none_rows = dfmeasures[dfmeasures['nameClean'].isna()]
    logger.debug(none_rows)
    if not none_rows.empty:
        logger.warning(f"Found {len(none_rows)} null nameClean. Filling with name.")
        dfmeasures['nameClean'] = dfmeasures['nameClean'].fillna(dfmeasures['name'])
        logger.info("Filled missing 'nameClean' values.")
    else:
        logger.info("No null 'nameClean' found.")
except Exception as e:
    logger.error("Failed to handle null values in 'nameClean'", exc_info=True)

# Drop the 'name' column after filling 'nameClean'
try:
    dfmeasures.drop('name', axis=1, inplace=True)
    logger.info("Dropped 'name' column.")
except Exception as e:
    logger.error("Failed to drop 'name' column", exc_info=True)

# Rename 'nameClean' to 'ing_name'
try:
    dfmeasures = dfmeasures.rename(columns={'nameClean': 'ing_name'})
    logger.info("Renamed 'nameClean' to 'ing_name'.")
    logger.debug(dfmeasures.info())
except Exception as e:
    logger.error("Failed to rename columns", exc_info=True)

#----------------------------------------------------------------------
#transforming the measures column

# Function to extract measure information from measures dict
def extract_measure(measures_dict, key_name):
  """
  Extracts a string value representing the measure from the given dictionary using a given name.
  Handles cases where the key name might be different.

  Args:
      measures_dict: A dictionary containing the measure information.
      key_name: The key name to look for.

  Returns:
      A string representing the measure in the format "amount unitShort".
  """
  if key_name in measures_dict:
    return f"{measures_dict[key_name]['amount']} {measures_dict[key_name]['unitShort']}"
  else:
    # Handle cases where the key might be different
    for key in measures_dict:
      if isinstance(measures_dict[key], dict):
        return extract_measure(measures_dict[key], key)
    # If no matching key is found, return an empty string
    return None

# Create two new columns with extracted measures
try:
    dfmeasures['measure_1'] = dfmeasures['measures'].apply(lambda x: extract_measure(x[0], "us"))
    dfmeasures['measure_2'] = dfmeasures['measures'].apply(lambda x: extract_measure(x[0], "metric"))
    logger.info("Created columns 'measure_1' and 'measure_2'.")
except Exception as e:
    logger.error("Failed to create 'measure_1' and 'measure_2' columns", exc_info=True)

# Drop the original 'measures' column
try:
    dfmeasures.drop('measures', axis=1, inplace=True)
    logger.info("Dropped 'measures' column.")
    logger.debug(dfmeasures.head())
    logger.debug(dfmeasures.info())
except Exception as e:
    logger.error("Failed to drop 'measures' column", exc_info=True)

# Function to combine measures from 'measure_1' and 'measure_2'
def combine_measures(row):
  """
  Combines values from 'measure_1' and 'measure_2' columns into a single string.

  Args:
      row: A pandas Series representing a row of the DataFrame.

  Returns:
      A string containing the combined measure value(s).
  """
  measure_1 = row['measure_1']
  measure_2 = row['measure_2']

  if measure_1 == measure_2:
    return measure_1  # Same values, return one
  else:
    return f"{measure_1} / {measure_2}"  # Different values are concatenated with "/"

# Apply the function to create a new 'measure' column
try:
    dfmeasures['measure'] = dfmeasures.apply(combine_measures, axis=1)
    logger.info("Created 'measure' column by combining 'measure_1' and 'measure_2'.")
except Exception as e:
    logger.error("Failed to create 'measure' column", exc_info=True)

# Drop 'measure_1' and 'measure_2' columns
try:
    dfmeasures.drop(['measure_1', 'measure_2'], axis=1, inplace=True)
    logger.info("Dropped 'measure_1' and 'measure_2' columns.")
    logger.debug(dfmeasures.head())
    logger.debug(dfmeasures.info())
except Exception as e:
    logger.error("Failed to drop 'measure_1' and 'measure_2' columns", exc_info=True)

#----------------------------------------------------------------------
# dealing with id_recipe in dfmeasures

# Create a mapping dictionary from dfrecipes, 
try:
    RECIPE_MAPPING = dfrecipes.set_index('recipe_title')['id_recipe'].to_dict()
    logger.info("Created 'RECIPE_MAPPING'.")
except Exception as e:
    logger.error("Failed to create 'RECIPE_MAPPING'", exc_info=True)

# Function to map recipe titles to id_recipe
def map_with_none(recipe_title, RECIPE_MAPPING):
    """Maps recipe names to id_recipe, handling missing values."""
    if pd.isna(recipe_title):
        return None  # Return None for missing titles
    return RECIPE_MAPPING.get(recipe_title, None)  # Use get() to avoid KeyError for missing keys

try:
    # Apply the mapping function to create 'id_recipe' column
    dfmeasures['id_recipe'] = dfmeasures['title'].apply(map_with_none, args=(RECIPE_MAPPING,))
    logger.info("Added 'id_recipe' column based on 'RECIPE_MAPPING'.")
    # Convert id_recipe to integer type where applicable, keeping None values
    dfmeasures['id_recipe'] = dfmeasures['id_recipe'].astype('Int64')
    logger.info("Converted 'id_recipe' column to Integer type.")
    logger.debug(dfmeasures.head())
    logger.debug(dfmeasures.info())
except Exception as e:
    logger.error("Failed to add or convert 'id_recipe' column", exc_info=True)

#----------------------------------------------------------------------
# dealing with id_ingredient in dfmeasures

# Create a mapping dictionary from dfIng
try:
    ING_MAPPING = dfIng.set_index('ing_name')['id_ingredient'].to_dict()
    logger.info("Created 'ING_MAPPING'.")
except Exception as e:
    logger.error("Failed to create 'ING_MAPPING'", exc_info=True)

# Function to map ingredient names to id_ingredient
def map_with_none(ing_name, ING_MAPPING):
    """Maps ingredient names to id_ingredient, handling missing values."""
    if pd.isna(ing_name):
        return None  # Return None for missing titles
    return ING_MAPPING.get(ing_name, None)  # Use get() to avoid KeyError for missing keys
    
try:
    # Apply the mapping function to create 'id_ingredient' column
    dfmeasures['id_ingredient'] = dfmeasures['ing_name'].apply(map_with_none, args=(ING_MAPPING,))
    logger.info("Added 'id_ingredient' column based on 'ING_MAPPING'.")
    # Convert id_ingredient to integer type where applicable, keeping None values
    dfmeasures['id_ingredient'] = dfmeasures['id_ingredient'].astype('Int64')
    logger.info("Converted 'id_ingredient' column to Integer type.")
    logger.debug(dfmeasures.head())
    logger.debug(dfmeasures.info())
except Exception as e:
    logger.error("Failed to add or convert 'id_ingredient' column", exc_info=True)

#----------------------------------------------------------------------
# Drop 'ing_name' and 'title' columns to finalize the reference DataFrame
try:
    dfreference_ing = dfmeasures.drop(['ing_name', 'title'], axis=1)
    logger.info("Dropped 'ing_name' and 'title' columns.")
    logger.debug(dfreference_ing.head())
    logger.debug(dfreference_ing.info())
except Exception as e:
    logger.error("Failed to finalize reference DataFrame by dropping 'ing_name' and 'title' columns", exc_info=True)
# ---------------------------------------------------------
try:
    # Find duplicates
    duplicates = dfreference_ing[dfreference_ing.duplicated()]
    logger.debug(duplicates)
    # Remove duplicates
    if not duplicates.empty:
        logger.warning(f"Found {len(duplicates)} duplicate reference ingredients. Removing duplicates.")
        dfreference_ing = dfreference_ing.drop_duplicates().reset_index(drop=True)
        logger.info(f"DataFrame now contains {len(dfreference_ing)} reference ingredients after removing duplicates.")
    else:
        logger.info("No duplicates found.")
except Exception as e:
    logger.error("Failed to handle duplicates", exc_info=True)

2024-07-09 00:01:23,069 - __main__ - INFO - Extracted ingredients and titles for 100 recipes.
2024-07-09 00:01:23,146 - __main__ - INFO - Constructed measures DataFrame with 1090 rows.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   extendedIngredients  100 non-null    object
 1   title                100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090 entries, 0 to 1089
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   nameClean  1082 non-null   object
 1   name       1090 non-null   object
 2   measures   1090 non-null   object
 3   title      1090 non-null   object
dtypes: object(4)
memory usage: 34.2+ KB


2024-07-09 00:01:23,181 - __main__ - WARNING - Found 8 null nameClean. Filling with name.
2024-07-09 00:01:23,186 - __main__ - INFO - Filled missing 'nameClean' values.
2024-07-09 00:01:23,188 - __main__ - INFO - Dropped 'name' column.
2024-07-09 00:01:23,191 - __main__ - INFO - Renamed 'nameClean' to 'ing_name'.
2024-07-09 00:01:23,208 - __main__ - INFO - Created columns 'measure_1' and 'measure_2'.
2024-07-09 00:01:23,210 - __main__ - INFO - Dropped 'measures' column.
2024-07-09 00:01:23,249 - __main__ - INFO - Created 'measure' column by combining 'measure_1' and 'measure_2'.
2024-07-09 00:01:23,252 - __main__ - INFO - Dropped 'measure_1' and 'measure_2' columns.
2024-07-09 00:01:23,269 - __main__ - INFO - Created 'RECIPE_MAPPING'.
2024-07-09 00:01:23,276 - __main__ - INFO - Added 'id_recipe' column based on 'RECIPE_MAPPING'.
2024-07-09 00:01:23,278 - __main__ - INFO - Converted 'id_recipe' column to Integer type.
2024-07-09 00:01:23,294 - __main__ - INFO - Created 'ING_MAPPING'.
20

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090 entries, 0 to 1089
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ing_name  1090 non-null   object
 1   measures  1090 non-null   object
 2   title     1090 non-null   object
dtypes: object(3)
memory usage: 25.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090 entries, 0 to 1089
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ing_name   1090 non-null   object
 1   title      1090 non-null   object
 2   measure_1  1090 non-null   object
 3   measure_2  1090 non-null   object
dtypes: object(4)
memory usage: 34.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090 entries, 0 to 1089
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ing_name  1090 non-null   object
 1   title     1090 non-null   object
 2   measure   1090 non-

2024-07-09 00:01:23,342 - __main__ - WARNING - Found 37 duplicate reference ingredients. Removing duplicates.
2024-07-09 00:01:23,345 - __main__ - INFO - DataFrame now contains 1053 reference ingredients after removing duplicates.


## Dealing with steps

In [11]:
# Extract the column that contains all the instructions along with the recipes titles
try:
    dfAllIns = pd.DataFrame(df, columns=['analyzedInstructions', 'title'])
    logger.info(f"Extracted instructions for {len(dfAllIns)} recipes.")
    logger.debug(dfAllIns.head())
    logger.debug(dfAllIns.info())
except Exception as e:
    logger.error("Failed to extract instructions and titles", exc_info=True)

#---------------------------------------------------------
# obtaining a new dataframe that contains steps of each recipe


# Define the required columns for the steps DataFrame
dfsteps_COLUMNS = ['steps', 'title']

# Initialize an empty list to collect rows of data
rows_data = []

# Define the required columns for the steps DataFrame
for i, row_series in dfAllIns.iterrows():
    try:
        # Access the recipe title for the current row 
        title = row_series['title']  # title is a string
        
        # Access the 'analyzedInstructions' column for the current row
        instructions_list = row_series['analyzedInstructions']  # instructions_list is a list containing one dictionary
        
        if instructions_list and isinstance(instructions_list[0], dict):  # Check if instructions_list is non-empty and its first element is a dictionary
            # Get steps list
            steps_list = instructions_list[0].get('steps', [])
        else:
            steps_list = None
        
        # Create a dictionary for the row data
        row_data = {
            'steps': steps_list,
            'title': title
        }
        
        # Append the row data to the list
        rows_data.append(row_data)
    except Exception as e:
        logger.error(f"Error processing row {i} for recipe '{title}'", exc_info=True)

# Create the DataFrame from the list of row data
try:
    dfsteps = pd.DataFrame(rows_data, columns=dfsteps_COLUMNS)
    logger.info(f"Constructed steps DataFrame for {len(dfsteps)} recipes.")
    logger.debug(dfsteps.head())
    logger.debug(dfsteps.info())
except Exception as e:
    logger.error("Failed to construct steps DataFrame", exc_info=True)


#---------------------------------------------------------
# transforming the steps column

# Define the required columns for the step DataFrame
dfsteps_COLUMNS = ['equipment', 'step', 'length', 'number', 'title']

# Initialize an empty list to collect rows of data
rows_data = []

# Iterate through each row of the original DataFrame dfsteps
for i, row_series in dfsteps.iterrows():
    try:
        # Access the recipe title for the current row 
        title = row_series['title']  # title is a string
        
        # Access the 'steps' column for the current row
        steps_list = row_series['steps']
        
        if steps_list is not None:
            # Iterate through each dictionary in the steps_list
            for each_dict in steps_list:
                if each_dict and isinstance(each_dict, dict):  # Check if each_dict is non-empty and is a dictionary
                    # Extract relevant information from each_dict
                    number = each_dict.get('number', None)
                    step = each_dict.get('step', None)
                    time = each_dict.get('length', {}).get('number', None)
                    unit = each_dict.get('length', {}).get('unit', None)
                    
                    # Calculate length based on time and unit
                    if time is None or unit is None:
                        length = None
                    else:
                        length = f"{time} {unit}"
                    
                    equipment_list = each_dict.get('equipment', [])  # Get equipment list or empty list if 'equipment' is missing
                    
                    # Handle NaN or None values in equipment_list
                    if isinstance(equipment_list, list):
                        equipment_list = [e if pd.notna(e) else None for e in equipment_list]
                    
                    # Create a dictionary for the row data
                    row_data = {
                        'length': length,
                        'number': number,
                        'step': step,
                        'equipment': equipment_list,
                        'title': title
                    }
                    
                    # Append the row data to the list
                    rows_data.append(row_data)
        else:
            # If steps_list is None, create a row with None values
            row_data = {
                'length': None,
                'number': None,
                'step': None,
                'equipment': None,
                'title': title
            }
            
            # Append the row data to the list
            rows_data.append(row_data)
    except Exception as e:
        logger.error(f"Error processing row {i} for recipe '{title}'", exc_info=True)

# Create the DataFrame from the list of row data
try:
    dfstep = pd.DataFrame(rows_data, columns=dfsteps_COLUMNS)
    logger.info(f"Constructed steps DataFrame with {len(dfstep)} steps.")
    logger.debug(dfstep.head())
    logger.debug(dfstep.info())
except Exception as e:
    logger.error("Failed to construct steps DataFrame", exc_info=True)

# Find duplicates
try:
    duplicates = dfstep[dfstep.duplicated(subset=["step", "length", "number", "title"])]
    logger.debug(duplicates)

    # Remove duplicates
    if not duplicates.empty:
        logger.warning(f"Found {len(duplicates)} duplicate steps. Removing duplicates.")
        dfstep = dfstep.drop_duplicates()
        dfstep.reset_index(drop=True, inplace=True)
        logger.info(f"DataFrame now contains {len(dfstep)} steps after removing duplicates.")
    else:
        logger.info("No duplicates found")
except Exception as e:
    logger.error("Failed to find or remove duplicates", exc_info=True)

# Checking if there's a Null values in step column
try:
    none_rows = dfstep[dfstep['step'].isna()]
    logger.debug(none_rows)

    # Deleting the null steps in dfstep DataFrame.
    if len(none_rows) > 0:
        logger.warning(f"Found {len(none_rows)} null steps. Deleting null steps.")
        dfstep.dropna(subset=['step'], inplace=True)
        dfstep.reset_index(drop=True, inplace=True)
        logger.info("Null steps deleted")
        logger.info(f"DataFrame now contains {len(dfstep)} steps after removing null values.")
    else:
        logger.info("No null steps found")
except Exception as e:
    logger.error("Failed to handle null steps", exc_info=True)

logger.debug(dfstep.head())
logger.debug(dfstep.info())   
    
#---------------------------------------------------------
#transforming equipment column

# Initialize an empty list to collect rows of data
rows_data = []

# Iterate through each row of the original DataFrame dfstep
for i, row_series in dfstep.iterrows():
    try:
        # Access the recipe title for the current row 
        title = row_series['title']
        # Access the step length for the current row 
        length = row_series['length']
        # Access the recipe step for the current row 
        step = row_series['step']
        # Access the step number for the current row 
        number = row_series['number']
        # Access the 'equipment' column for the current row
        equipments_list = row_series['equipment']  #
        
        # Initialize variables to store processed data
        equipments_name_list = []
        
        # Process equipments_list
        if equipments_list and isinstance(equipments_list, list):
            for each_dict in equipments_list:
                if each_dict and isinstance(each_dict, dict):  # Check if each_dict is non-empty and is a dictionary
                    # Get a list containing equipments names
                    equipment_name = each_dict.get('name', None)
                    if equipment_name is not None:  # Ensure equipment_name is not None
                        equipments_name_list.append(equipment_name)
        else:
            equipments_name_list = None  # Handle case where equipments_list is None or not a list
        
        # Create a dictionary for the row data
        row_data = {
            'length': length,
            'number': number,
            'step': step,
            'equipment': equipments_name_list,
            'title': title
        }
        
        # Append the row data to the list
        rows_data.append(row_data)
    except Exception as e:
        logger.error(f"Error processing row {i} for recipe '{title}'", exc_info=True)

# Create the DataFrame from the list of row data
try:
    dfstepclean = pd.DataFrame(rows_data, columns=dfsteps_COLUMNS)
    logger.info(f"Constructed dfstepclean DataFrame with {len(dfstepclean)} steps and contains cleaned lists of equipments")
    logger.debug(dfstepclean.head())
    logger.debug(dfstepclean.info())
except Exception as e:
    logger.error("Failed to construct dfstepclean DataFrame", exc_info=True)

#---------------------------------------------------------
#dealing with dfstep_final

try:
    # generating id_step
    dfstepclean['id_step'] = range(counter, counter + len(dfstepclean))
    logger.info("Added 'id_step'.")
    logger.debug(dfstepclean.info())

    dfstep_final = dfstepclean.copy()
    logger.debug(dfstep_final.info())

    # Apply the custom map function
    dfstep_final['id_recipe'] = dfstep_final['title'].apply(map_with_none, args=(RECIPE_MAPPING,))
    logger.info("'id_recipe' column added according to the 'RECIPE_MAPPING'")

    # Convert id_recipe to integer type where applicable, keeping None values
    dfstep_final['id_recipe'] = dfstep_final['id_recipe'].astype('Int64')
    logger.info("'id_recipe' column converted to Integer")

    # Drop columns by label (column name)
    dfstep_final.drop(columns=['equipment', 'title'], inplace=True)
    logger.info("'equipment' and 'title' columns deleted")
    logger.debug(dfstep_final.head())
    logger.debug(dfstep_final.info())
except Exception as e:
    logger.error("Failed to finalize dfstep_final DataFrame", exc_info=True)


2024-07-09 00:01:23,402 - __main__ - INFO - Extracted instructions for 100 recipes.
2024-07-09 00:01:23,435 - __main__ - INFO - Constructed steps DataFrame for 100 recipes.
2024-07-09 00:01:23,496 - __main__ - INFO - Constructed steps DataFrame with 629 steps.
2024-07-09 00:01:23,511 - __main__ - INFO - No duplicates found
2024-07-09 00:01:23,516 - __main__ - WARNING - Found 3 null steps. Deleting null steps.
2024-07-09 00:01:23,519 - __main__ - INFO - Null steps deleted
2024-07-09 00:01:23,520 - __main__ - INFO - DataFrame now contains 626 steps after removing null values.
2024-07-09 00:01:23,595 - __main__ - INFO - Constructed dfstepclean DataFrame with 626 steps and contains cleaned lists of equipments
2024-07-09 00:01:23,609 - __main__ - INFO - Added 'id_step'.
2024-07-09 00:01:23,627 - __main__ - INFO - 'id_recipe' column added according to the 'RECIPE_MAPPING'


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   analyzedInstructions  100 non-null    object
 1   title                 100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   steps   97 non-null     object
 1   title   100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629 entries, 0 to 628
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   equipment  626 non-null    object 
 1   step       626 non-null    object 
 2   length     154 non-null    object 
 3   number     626 non-null    float64
 4   title      629 non-nu

2024-07-09 00:01:23,630 - __main__ - INFO - 'id_recipe' column converted to Integer
2024-07-09 00:01:23,633 - __main__ - INFO - 'equipment' and 'title' columns deleted


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626 entries, 0 to 625
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   step       626 non-null    object 
 1   length     154 non-null    object 
 2   number     626 non-null    float64
 3   id_step    626 non-null    int64  
 4   id_recipe  626 non-null    Int64  
dtypes: Int64(1), float64(1), int64(1), object(2)
memory usage: 25.2+ KB


## Dealing with instructions

In [12]:
try:
    # Assign a unique number to each unique title
    dfstep_final['instruction_id'] = pd.factorize(dfstep_final['id_recipe'])[0] + counter
    logger.info("Added 'instrdfstep_final.head()ction_id'.")
    logger.debug(dfstep_final.head(30))
    logger.debug(dfstep_final.info())
except Exception as e:
    logger.error("Failed to add 'instruction_id'.", exc_info=True)


try:
    # Create the instructions dataframe
    dfIns = dfstep_final[['instruction_id', 'id_recipe']].copy()
    logger.info("Constructed instructions dataframe")
    logger.debug(dfIns.head())
    logger.debug(dfIns.info())
except Exception as e:
    logger.error("Failed to construct instructions dataframe", exc_info=True)

try:
    # Find duplicates
    duplicates = dfIns[dfIns.duplicated()]
    logger.debug(duplicates)

    # Remove duplicates
    if not duplicates.empty:
        logger.warning(f"Found {len(duplicates)} duplicate instructions. Removing duplicates.")
        dfIns = dfIns.drop_duplicates()
        dfIns.reset_index(drop=True, inplace=True)
        logger.info(f"DataFrame now contains {len(dfIns)} instructions after removing duplicates.")
    else:
        logger.info("No duplicates found")
except Exception as e:
    logger.error("An unexpected error occurred while handling duplicates.", exc_info=True)

try:
    # Drop a column by label (column name)
    dfstep_final.drop(columns=['id_recipe'], inplace=True)
    logger.info("'id_recipe' deleted from steps dataframe")
except Exception as e:
    logger.error("Failed to drop 'id_recipe' column.", exc_info=True)

logger.debug(dfIns.head())
logger.debug(dfIns.info())
logger.debug(dfstep_final.info())

2024-07-09 00:01:23,662 - __main__ - INFO - Added 'instrdfstep_final.head()ction_id'.
2024-07-09 00:01:23,686 - __main__ - INFO - Constructed instructions dataframe
2024-07-09 00:01:23,699 - __main__ - WARNING - Found 529 duplicate instructions. Removing duplicates.
2024-07-09 00:01:23,702 - __main__ - INFO - DataFrame now contains 97 instructions after removing duplicates.
2024-07-09 00:01:23,704 - __main__ - INFO - 'id_recipe' deleted from steps dataframe


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626 entries, 0 to 625
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   step            626 non-null    object 
 1   length          154 non-null    object 
 2   number          626 non-null    float64
 3   id_step         626 non-null    int64  
 4   id_recipe       626 non-null    Int64  
 5   instruction_id  626 non-null    int64  
dtypes: Int64(1), float64(1), int64(2), object(2)
memory usage: 30.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626 entries, 0 to 625
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   instruction_id  626 non-null    int64
 1   id_recipe       626 non-null    Int64
dtypes: Int64(1), int64(1)
memory usage: 10.5 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 2 columns):
 #   Column          Non-Null Count 

## Dealing with equipments

In [13]:
# Define the required columns for the equipments DataFrame
dfequip_COLUMNS = ['name']

# Initialize an empty list to collect rows of data
rows_data = []

# Iterate through each row of the original DataFrame
for i, row_series in dfstep.iterrows():
    try:
        equip_list = row_series['equipment']
        
        if equip_list and isinstance(equip_list, list):
            for each_dict in equip_list:
                # Check if the element is a dictionary
                if each_dict and isinstance(each_dict, dict):
                    name = each_dict.get('name', None)
                    
                    # Create a dictionary for the row data
                    row_data = {
                        'name': name
                    }
                    
                    # Append the row data to the list
                    rows_data.append(row_data)
                else:
                    logger.warning(f"Non-dictionary element found in equipment list at row {i}. Skipping element.")
        else:
            logger.warning(f"Equipment list is empty or not a list at row {i}. Skipping row.")

    except Exception as e:
        logger.error(f"Error processing row {i}", exc_info=True)

# Create the DataFrame from the list of row data
try:
    dfequip = pd.DataFrame(rows_data, columns=dfequip_COLUMNS)
    logger.info(f"Constructed equipments DataFrame with {len(dfequip)} equipments.")
    logger.debug(dfequip.head())
    logger.debug(dfequip.info())
except Exception as e:
    logger.error("Failed to construct equipments DataFrame.", exc_info=True)
    dfequip = pd.DataFrame(columns=dfequip_COLUMNS)  # Create an empty DataFrame as a fallback


try:
    # Find duplicates
    duplicates = dfequip[dfequip.duplicated()]
    logger.debug(duplicates)

    # Remove duplicates
    if not duplicates.empty:
        logger.warning(f"Found {len(duplicates)} duplicate equipments. Removing duplicates.")
        dfequip = dfequip.drop_duplicates()
        dfequip.reset_index(drop=True, inplace=True)
        logger.info(f"DataFrame now contains {len(dfequip)} equipments after removing duplicates.")
    else:
        logger.info("No duplicates found")
except Exception as e:
    logger.error("An error occurred while finding or removing duplicates.", exc_info=True)



# Generate 'id_equipment' column
try:
    dfequip['id_equipment'] = range(counter, counter +len(dfequip))
    logger.info("Added 'id_equipment'.")
    logger.debug(dfequip.info())
except Exception as e:
    logger.error("Failed to add 'id_equipment' column.", exc_info=True)

# Rename 'name' column to 'equip_name'
try:
    dfequip = dfequip.rename(columns={'name': 'equip_name'})
    logger.info("Renamed column 'name' to 'equip_name'.")
    logger.debug(dfequip.head())
    logger.debug(dfequip.info())
except KeyError as e:
    logger.error("Failed to rename 'name' column because it is missing.", exc_info=True)
except Exception as e:
    logger.error("An unexpected error occurred while renaming 'name' column.", exc_info=True)


2024-07-09 00:01:23,742 - __main__ - WARNING - Equipment list is empty or not a list at row 2. Skipping row.
2024-07-09 00:01:23,744 - __main__ - WARNING - Equipment list is empty or not a list at row 3. Skipping row.
2024-07-09 00:01:23,747 - __main__ - WARNING - Equipment list is empty or not a list at row 5. Skipping row.
2024-07-09 00:01:23,748 - __main__ - WARNING - Equipment list is empty or not a list at row 6. Skipping row.
2024-07-09 00:01:23,750 - __main__ - WARNING - Equipment list is empty or not a list at row 9. Skipping row.
2024-07-09 00:01:23,751 - __main__ - WARNING - Equipment list is empty or not a list at row 11. Skipping row.
2024-07-09 00:01:23,754 - __main__ - WARNING - Equipment list is empty or not a list at row 12. Skipping row.
2024-07-09 00:01:23,756 - __main__ - WARNING - Equipment list is empty or not a list at row 13. Skipping row.
2024-07-09 00:01:23,758 - __main__ - WARNING - Equipment list is empty or not a list at row 15. Skipping row.
2024-07-09 00:0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    420 non-null    object
dtypes: object(1)
memory usage: 3.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          48 non-null     object
 1   id_equipment  48 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 896.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   equip_name    48 non-null     object
 1   id_equipment  48 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 896.0+ bytes


## Dealing with reference_equip

In [14]:
# Explode the equipment column in dfstepclean dataframe because equipment is a list of equipments 
try:
    dfstepclean_exploded = dfstepclean.explode(['equipment'])
    dfstepclean_exploded = dfstepclean_exploded.reset_index(drop=True)
    logger.info("'equipment' column in step dataframe exploded.")
except Exception as e:
    logger.error("Failed to explode 'equipment' column.", exc_info=True)

# Rename the exploded column to ing_name
try:
    dfstepclean_exploded = dfstepclean_exploded.rename(columns={'equipment': 'equip_name'})
    logger.info("Renamed column 'equipment' to 'equip_name'.")
    logger.debug(dfstepclean_exploded.head())
except Exception as e:
    logger.error("Failed to rename 'equipment' column", exc_info=True)

# -----------------------------------------------------------
# dealing with id_equipment in dfstepclean_exploded
try:
    # Create a mapping dictionary from dfequip
    EQUIP_MAPPING = dfequip.set_index('equip_name')['id_equipment'].to_dict()
    logger.info("'EQUIP_MAPPING' created")
except Exception as e:
    logger.error("Failed to create 'EQUIP_MAPPING'.", exc_info=True)

# Function to map equipment names to id_equipment, handling missing values
def map_with_none(equip_name, EQUIP_MAPPING):
    """Maps equipment names to id_equipment, handling missing values."""
    if pd.isna(equip_name):
        return None  # Return None for missing equipment names
    return EQUIP_MAPPING.get(equip_name, None)  # Use get() to avoid KeyError for missing keys

# Apply the custom map function to fill the new column
try:
    dfstepclean_exploded['id_equipment'] = dfstepclean_exploded['equip_name'].apply(map_with_none, args=(EQUIP_MAPPING,))
    logger.info("'id_equipment' column added according to the 'EQUIP_MAPPING'.")
except Exception as e:
    logger.error("Failed to add 'id_equipment' column.", exc_info=True)

# Convert id_equipment to integer type where applicable, keeping None values
try:
    dfstepclean_exploded['id_equipment'] = dfstepclean_exploded['id_equipment'].astype('Int64')
    logger.info("'id_equipment' column converted to Integer.")
    logger.debug(dfstepclean_exploded.head())
    logger.debug(dfstepclean_exploded.info())
except Exception as e:
    logger.error("Failed to convert 'id_equipment' column to Integer.", exc_info=True)

# -----------------------------------------------------------
# dealing with id_recipe in dfstepclean_exploded

try:
    # Apply the custom map function to fill the new column
    dfstepclean_exploded['id_recipe'] = dfstepclean_exploded['title'].apply(map_with_none, args=(RECIPE_MAPPING,))
    logger.info("'id_recipe' column added according to the 'RECIPE_MAPPING'.")
except Exception as e:
    logger.error("Failed to add 'id_recipe' column.", exc_info=True)
    
try:
    # Convert id_recipe to integer type where applicable, keeping None values
    dfstepclean_exploded['id_recipe'] = dfstepclean_exploded['id_recipe'].astype('Int64')
    logger.info("'id_recipe' column converted to Integer.")
    logger.debug(dfstepclean_exploded.head())
    logger.debug(dfstepclean_exploded.info())
except Exception as e:
    logger.error("Failed to convert 'id_recipe' column to Integer.", exc_info=True)

# -----------------------------------------------------------
# creating the dfreference_equip dataframe
try:
    dfreference_equip = dfstepclean_exploded[['id_recipe', 'id_step', 'id_equipment']]
    logger.info("Constructed reference_equip dataframe.")
    logger.debug(dfreference_equip.head())
    logger.debug(dfreference_equip.info())
except Exception as e:
    logger.error("Failed to create 'dfreference_equip' dataframe.", exc_info=True)

# Clean up dataframes
try:
    del dfstepclean_exploded
    del dfstepclean
    logger.info("Deleted intermediate dataframes.")
except Exception as e:
    logger.error("One or more intermediate dataframes were not found during deletion.", exc_info=True)


try:
    # Find duplicates
    duplicates = dfreference_equip[dfreference_equip.duplicated()]
    logger.debug(duplicates)

    # Remove duplicates
    if not duplicates.empty:
        logger.warning(f"Found {len(duplicates)} duplicate refrence equipment. Removing duplicates.")
        dfreference_equip = dfreference_equip.drop_duplicates()
        dfreference_equip.reset_index(drop=True, inplace=True)
        logger.info(f"DataFrame now contains {len(dfreference_equip)} refrence equipment after removing duplicates.")
    else:
        logger.info("No duplicates found")
except Exception as e:
    logger.error("An error occurred while finding or removing duplicates.", exc_info=True)
  


2024-07-09 00:01:24,469 - __main__ - INFO - 'equipment' column in step dataframe exploded.
2024-07-09 00:01:24,471 - __main__ - INFO - Renamed column 'equipment' to 'equip_name'.
2024-07-09 00:01:24,478 - __main__ - INFO - 'EQUIP_MAPPING' created
2024-07-09 00:01:24,484 - __main__ - INFO - 'id_equipment' column added according to the 'EQUIP_MAPPING'.
2024-07-09 00:01:24,488 - __main__ - INFO - 'id_equipment' column converted to Integer.
2024-07-09 00:01:24,502 - __main__ - INFO - 'id_recipe' column added according to the 'RECIPE_MAPPING'.
2024-07-09 00:01:24,503 - __main__ - INFO - 'id_recipe' column converted to Integer.
2024-07-09 00:01:24,515 - __main__ - INFO - Constructed reference_equip dataframe.
2024-07-09 00:01:24,525 - __main__ - INFO - Deleted intermediate dataframes.
2024-07-09 00:01:24,532 - __main__ - INFO - No duplicates found


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755 entries, 0 to 754
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   equip_name    420 non-null    object 
 1   step          755 non-null    object 
 2   length        193 non-null    object 
 3   number        755 non-null    float64
 4   title         755 non-null    object 
 5   id_step       755 non-null    int64  
 6   id_equipment  420 non-null    Int64  
dtypes: Int64(1), float64(1), int64(1), object(4)
memory usage: 42.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755 entries, 0 to 754
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   equip_name    420 non-null    object 
 1   step          755 non-null    object 
 2   length        193 non-null    object 
 3   number        755 non-null    float64
 4   title         755 non-null    object 
 5   id_step       755 non-null

## Dealing with dish types

In [15]:
# Define the required columns for the dish DataFrame
dfdish_COLUMNS = ['dishTypes', 'title']
# Construct the DataFrame
try:
    dfdish = pd.DataFrame(df, columns=dfdish_COLUMNS)
    logger.info(f"Extracted dish types for {len(dfdish)} recipes.")
    logger.debug(dfdish.head())
    logger.debug(dfdish.info())
except Exception as e:
    logger.error("Failed to construct dfdish DataFrame.", exc_info=True)

# Explode the 'dishTypes' column
try:
    dfALLdish_types = dfdish.explode('dishTypes').reset_index(drop=True)
    logger.info(f"'dishTypes' column exploded and there are {len(dfALLdish_types)} dish types.")
    logger.debug(dfALLdish_types.head())
    logger.debug(dfALLdish_types.info())
except Exception as e:
    logger.error("Failed to explode 'dishTypes' column.", exc_info=True)

# Copy the exploded DataFrame
dfdish_type=dfALLdish_types.copy()

# Delete 'title' column
try:
    dfdish_type.drop(columns=['title'], inplace=True)
    logger.info("Deleted 'title' column.")
except Exception as e:
    logger.error("Failed to delete 'title' column.", exc_info=True)

try:
    # Find duplicates
    duplicates = dfdish_type[dfdish_type.duplicated()]
    logger.debug(duplicates)
    
    # Remove duplicates
    if not duplicates.empty:
        logger.warning(f"Found {len(duplicates)} duplicate dish types. Removing duplicates.")
        dfdish_type = dfdish_type.drop_duplicates()
        dfdish_type.reset_index(drop=True, inplace=True)
        logger.info(f"DataFrame now contains {len(dfdish_type)} dish types after removing duplicates.")
    else:
        logger.info("No duplicates found.")
except Exception as e:
    logger.error("An error occurred while finding or removing duplicates.", exc_info=True)


try:
    # Find Null values
    none_rows = dfdish_type[dfdish_type['dishTypes'].isna()]
    logger.debug(none_rows)

    # Remove null values
    if not none_rows.empty:
        logger.warning(f"Found {len(none_rows)} null dish types. Removing null values.")
        dfdish_type.dropna(inplace=True)
        dfdish_type.reset_index(drop=True, inplace=True)
        logger.info(f"DataFrame now contains {len(dfdish_type)} dish types after removing null values.")
    else:
        logger.info("No null values found.")
except Exception as e:
    logger.error("An error occurred while finding or removing null values.", exc_info=True)

# Rename 'dishTypes' column to 'dish_type'
try:
    dfdish_type = dfdish_type.rename(columns={'dishTypes': 'dish_type'})
    logger.info("Renamed column 'dishTypes' to 'dish_type'.")
except Exception as e:
    logger.error("Failed to rename 'dishTypes' column.", exc_info=True)

# Generate 'id_dish_type' column
try:
    dfdish_type['id_dish_type'] = range(counter, counter +len(dfdish_type))
    logger.info("Added 'id_dish_type' column.")
    logger.debug(dfdish_type.head())
    logger.debug(dfdish_type.info())
except Exception as e:
    logger.error("Failed to add 'id_dish_type' column.", exc_info=True)


2024-07-09 00:01:24,554 - __main__ - INFO - Extracted dish types for 100 recipes.
2024-07-09 00:01:24,571 - __main__ - INFO - 'dishTypes' column exploded and there are 355 dish types.
2024-07-09 00:01:24,579 - __main__ - INFO - Deleted 'title' column.
2024-07-09 00:01:24,584 - __main__ - WARNING - Found 329 duplicate dish types. Removing duplicates.
2024-07-09 00:01:24,585 - __main__ - INFO - DataFrame now contains 26 dish types after removing duplicates.
2024-07-09 00:01:24,590 - __main__ - WARNING - Found 1 null dish types. Removing null values.
2024-07-09 00:01:24,592 - __main__ - INFO - DataFrame now contains 25 dish types after removing null values.
2024-07-09 00:01:24,594 - __main__ - INFO - Renamed column 'dishTypes' to 'dish_type'.
2024-07-09 00:01:24,598 - __main__ - INFO - Added 'id_dish_type' column.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   dishTypes  100 non-null    object
 1   title      100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355 entries, 0 to 354
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   dishTypes  352 non-null    object
 1   title      355 non-null    object
dtypes: object(2)
memory usage: 5.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   dish_type     25 non-null     object
 1   id_dish_type  25 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 528.0+ bytes


## dfis_a dataframe

In [16]:
# Create a mapping dictionary from dfdish_type
try:
    DISH_MAPPING = dfdish_type.set_index('dish_type')['id_dish_type'].to_dict()
    logger.info("'DISH_MAPPING' created.")
except Exception as e:
    logger.error("Failed to create DISH_MAPPING.", exc_info=True)

# Define a function to map dish types to id_dish_type, handling missing values
def map_with_none(dish_type, mapping_dict):
    """Maps dish type names to id_dish_type, handling missing values."""
    if pd.isna(dish_type):
        return None  # Return None for missing dish types
    return mapping_dict.get(dish_type, None)  # Use get() to avoid KeyError for missing keys

# Apply the custom map function to 'dishTypes' column
try:
    dfALLdish_types['id_dish_type'] = dfALLdish_types['dishTypes'].apply(map_with_none, args=(DISH_MAPPING,))
    logger.info("'id_dish_type' column added according to the 'DISH_MAPPING'.")
except Exception as e:
    logger.error("Failed to map 'dishTypes' to 'id_dish_type'.", exc_info=True)

# Convert 'id_dish_type' to integer type where applicable, keeping None values
try:
    dfALLdish_types['id_dish_type'] = dfALLdish_types['id_dish_type'].astype('Int64')
    logger.info("'id_dish_type' column converted to Integer.")
except Exception as e:
    logger.error("Failed to convert 'id_dish_type' to Integer.", exc_info=True)

# Apply the custom map function to 'title' column for 'id_recipe'
try:
    dfALLdish_types['id_recipe'] = dfALLdish_types['title'].apply(map_with_none, args=(RECIPE_MAPPING,))
    logger.info("'id_recipe' column added according to the 'RECIPE_MAPPING'.")
except Exception as e:
    logger.error("Failed to map 'title' to 'id_recipe'.", exc_info=True)

# Convert 'id_recipe' to integer type where applicable, keeping None values
try:
    dfALLdish_types['id_recipe'] = dfALLdish_types['id_recipe'].astype('Int64')
    logger.info("'id_recipe' column converted to Integer.")
except Exception as e:
    logger.error("Failed to convert 'id_recipe' to Integer.", exc_info=True)

# Log the DataFrame information
logger.debug(dfALLdish_types.head())
logger.debug(dfALLdish_types.info())

# Delete 'dishTypes' and 'title' columns
try:
    dfALLdish_types.drop(columns=['dishTypes', 'title'], inplace=True)
    logger.info("Deleted columns 'dishTypes' and 'title'.")
except Exception as e:
    logger.error("Failed to delete columns 'dishTypes' or 'title'.", exc_info=True)

# Copy the DataFrame and delete the original
try:
    dfis_a = dfALLdish_types.copy()
    del dfALLdish_types
    logger.info("Copied dfALLdish_types to dfis_a and deleted dfALLdish_types.")
except Exception as e:
    logger.error("Failed to copy or delete dfALLdish_types.", exc_info=True)

# Log the final DataFrame information
logger.debug(dfis_a.head())
logger.debug(dfis_a.info())


2024-07-09 00:01:24,622 - __main__ - INFO - 'DISH_MAPPING' created.
2024-07-09 00:01:24,628 - __main__ - INFO - 'id_dish_type' column added according to the 'DISH_MAPPING'.
2024-07-09 00:01:24,630 - __main__ - INFO - 'id_dish_type' column converted to Integer.
2024-07-09 00:01:24,636 - __main__ - INFO - 'id_recipe' column added according to the 'RECIPE_MAPPING'.
2024-07-09 00:01:24,638 - __main__ - INFO - 'id_recipe' column converted to Integer.
2024-07-09 00:01:24,648 - __main__ - INFO - Deleted columns 'dishTypes' and 'title'.
2024-07-09 00:01:24,650 - __main__ - INFO - Copied dfALLdish_types to dfis_a and deleted dfALLdish_types.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355 entries, 0 to 354
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   dishTypes     352 non-null    object
 1   title         355 non-null    object
 2   id_dish_type  352 non-null    Int64 
 3   id_recipe     355 non-null    Int64 
dtypes: Int64(2), object(2)
memory usage: 11.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355 entries, 0 to 354
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   id_dish_type  352 non-null    Int64
 1   id_recipe     355 non-null    Int64
dtypes: Int64(2)
memory usage: 6.4 KB


## Dealing with cuisines

### extracting needed columns

In [17]:
# Define the required columns for the cuisines DataFrame
dfcuisines_COLUMNS = ['cuisines', 'title']

# Construct the DataFrame
try:
    dfALLcuisines = pd.DataFrame(df, columns=dfcuisines_COLUMNS)
    logger.info(f"Extracted cuisines for {len(dfALLcuisines)} recipes.")
    logger.debug(dfALLcuisines.head())
    logger.debug(dfALLcuisines.info())
except Exception as e:
    logger.error("Failed to construct dfALLcuisines DataFrame.", exc_info=True)

# Explode the 'cuisines' column
try:
    dfALLcuisines = dfALLcuisines.explode('cuisines').reset_index(drop=True)
    logger.info(f"'cuisines' column exploded and contains {len(dfALLcuisines)} cuisines.")
    logger.debug(dfALLcuisines.info())
except Exception as e:
    logger.error("Failed to explode 'cuisines' column.", exc_info=True)

# Copy the exploded DataFrame
dfcuisine=dfALLcuisines.copy()

# Delete 'title' column
try:
    dfcuisine.drop(columns=['title'], inplace=True)
    logger.info("Deleted 'title' column.")
except Exception as e:
    logger.error("Failed to delete 'title' column.", exc_info=True)

try:
    # Find duplicates
    duplicates = dfcuisine[dfcuisine.duplicated()]
    logger.debug(duplicates)

    # Remove duplicates
    if not duplicates.empty:
        logger.warning(f"Found {len(duplicates)} duplicate cuisines. Removing duplicates.")
        dfcuisine = dfcuisine.drop_duplicates()
        dfcuisine.reset_index(drop=True, inplace=True)
        logger.info(f"DataFrame now contains {len(dfcuisine)} cuisines after removing duplicates.")
    else:
        logger.info("No duplicates found.")
except Exception as e:
    logger.error("An error occurred while finding or removing duplicates.", exc_info=True)

try:
    # Find Null values
    none_rows = dfcuisine[dfcuisine['cuisines'].isna()]
    logger.debug(none_rows)

    # Remove null values
    if not none_rows.empty:
        logger.warning(f"Found {len(none_rows)} null cuisine values. Removing null values.")
        dfcuisine.dropna(inplace=True)
        dfcuisine.reset_index(drop=True, inplace=True)
        logger.info(f"DataFrame now contains {len(dfcuisine)} cuisines after removing null values.")
    else:
        logger.info("No null values found.")
except Exception as e:
    logger.error("An error occurred while finding or removing null values.", exc_info=True)

# Rename 'cuisines' column to 'recipe_cuisine'
try:
    dfcuisine = dfcuisine.rename(columns={'cuisines': 'recipe_cuisine'})
    logger.info("Renamed column 'cuisines' to 'recipe_cuisine'.")
    logger.debug(dfcuisine.info())
except Exception as e:
    logger.error("Failed to rename 'cuisines' column.", exc_info=True)

# Generate 'id_cuisine' column
try:
    dfcuisine['id_cuisine'] = range(counter, counter + len(dfcuisine))
    logger.info("Added 'id_cuisine' column.")
    logger.debug(dfcuisine.head())
    logger.debug(dfcuisine.info())
except Exception as e:
    logger.error("Failed to add 'id_cuisine' column.", exc_info=True)


2024-07-09 00:01:24,686 - __main__ - INFO - Extracted cuisines for 100 recipes.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cuisines  100 non-null    object
 1   title     100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


2024-07-09 00:01:24,704 - __main__ - INFO - 'cuisines' column exploded and contains 126 cuisines.
2024-07-09 00:01:24,711 - __main__ - INFO - Deleted 'title' column.
2024-07-09 00:01:24,716 - __main__ - WARNING - Found 107 duplicate cuisines. Removing duplicates.
2024-07-09 00:01:24,720 - __main__ - INFO - DataFrame now contains 19 cuisines after removing duplicates.
2024-07-09 00:01:24,723 - __main__ - WARNING - Found 1 null cuisine values. Removing null values.
2024-07-09 00:01:24,725 - __main__ - INFO - DataFrame now contains 18 cuisines after removing null values.
2024-07-09 00:01:24,727 - __main__ - INFO - Renamed column 'cuisines' to 'recipe_cuisine'.
2024-07-09 00:01:24,734 - __main__ - INFO - Added 'id_cuisine' column.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cuisines  57 non-null     object
 1   title     126 non-null    object
dtypes: object(2)
memory usage: 2.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   recipe_cuisine  18 non-null     object
dtypes: object(1)
memory usage: 272.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   recipe_cuisine  18 non-null     object
 1   id_cuisine      18 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 416.0+ bytes


### dfbelongs dataframe

In [18]:
# Ensure the CUISINE_MAPPING dictionary is created
try:
    CUISINE_MAPPING = dfcuisine.set_index('recipe_cuisine')['id_cuisine'].to_dict()
    logger.info("'CUISINE_MAPPING' created")
except Exception as e:
    logger.error("Failed to create 'CUISINE_MAPPING'.", exc_info=True)

# Define a function to map recipe cuisines to id_cuisine, handling missing values
def map_with_none(recipe_cuisine, CUISINE_MAPPING):
    """Maps recipes cuisines to id_cuisine, handling missing values."""
    if pd.isna(recipe_cuisine):
        return None  # Return None for missing cuisines
    return CUISINE_MAPPING.get(recipe_cuisine, None)  # Use get() to avoid KeyError for missing keys

# Apply the custom map function to add 'id_cuisine' column
try:
    dfALLcuisines['id_cuisine'] = dfALLcuisines['cuisines'].apply(map_with_none, args=(CUISINE_MAPPING,))
    logger.info("'id_cuisine' column added according to the 'CUISINE_MAPPING'.")
except Exception as e:
    logger.error("Failed to map 'id_cuisine' to 'cuisines'.", exc_info=True)

# Convert id_cuisine to integer type where applicable, keeping None values
try:
    dfALLcuisines['id_cuisine'] = dfALLcuisines['id_cuisine'].astype('Int64')
    logger.info("'id_cuisine' column converted to Integer ")
except Exception as e:
    logger.error("Failed to convert 'id_cuisine' to Integer.", exc_info=True)


# Apply the custom map function to add 'id_recipe' column
try:
    dfALLcuisines['id_recipe'] = dfALLcuisines['title'].apply(map_with_none, args=(RECIPE_MAPPING,))
    logger.info("'id_recipe' column added according to the 'RECIPE_MAPPING'.")
except Exception as e:
    logger.error("Failed to map 'id_recipe' to 'title'.", exc_info=True)

# Convert id_recipe to integer type where applicable, keeping None values
try:
    dfALLcuisines['id_recipe'] = dfALLcuisines['id_recipe'].astype('Int64')
    logger.info("'id_recipe' column converted to Integer.")
except Exception as e:
    logger.error("Failed to convert 'id_recipe' to Integer.", exc_info=True)

# Log the DataFrame head and info
logger.debug(dfALLcuisines.head())
logger.debug(dfALLcuisines.info())

# Drop 'cuisines' and 'title' columns from dfcuisines
try:
    dfALLcuisines.drop(columns=['cuisines', 'title'], inplace=True)
    logger.info("Deleted 'cuisines' and 'title' columns.")
except Exception as e:
    logger.error("Failed to delete 'cuisines' or 'title' columns.", exc_info=True)

# Copy dfcuisines to dfbelongs and delete dfcuisines
try:
    dfbelongs = dfALLcuisines.copy()
    logger.info("Copied dfALLcuisines to dfbelongs.")
    
    del dfALLcuisines
    logger.info("Deleted dfALLcuisines.")

    logger.debug(dfbelongs.head())
    logger.debug(dfbelongs.info())
except Exception as e:
    logger.error("Failed to copy or delete dfcuisines.", exc_info=True)


2024-07-09 00:01:24,763 - __main__ - INFO - 'CUISINE_MAPPING' created
2024-07-09 00:01:24,766 - __main__ - INFO - 'id_cuisine' column added according to the 'CUISINE_MAPPING'.
2024-07-09 00:01:24,771 - __main__ - INFO - 'id_cuisine' column converted to Integer 
2024-07-09 00:01:24,774 - __main__ - INFO - 'id_recipe' column added according to the 'RECIPE_MAPPING'.
2024-07-09 00:01:24,777 - __main__ - INFO - 'id_recipe' column converted to Integer.
2024-07-09 00:01:24,789 - __main__ - INFO - Deleted 'cuisines' and 'title' columns.
2024-07-09 00:01:24,790 - __main__ - INFO - Copied dfALLcuisines to dfbelongs.
2024-07-09 00:01:24,791 - __main__ - INFO - Deleted dfALLcuisines.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   cuisines    57 non-null     object
 1   title       126 non-null    object
 2   id_cuisine  57 non-null     Int64 
 3   id_recipe   126 non-null    Int64 
dtypes: Int64(2), object(2)
memory usage: 4.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   id_cuisine  57 non-null     Int64
 1   id_recipe   126 non-null    Int64
dtypes: Int64(2)
memory usage: 2.3 KB


## Transform Recap

In [19]:
logger.info("Displaying transforamtion recap")

2024-07-09 00:01:24,820 - __main__ - INFO - Displaying transforamtion recap


### recipe table
(id_recipe INT, recipe_title VARCHAR(50), ready_min INT, summary VARCHAR(2000), servings INT, is_cheap LOGICAL, price_per_serving DOUBLE, is_vegetarian LOGICAL, is_vegan LOGICAL, is_glutenFree LOGICAL, is_dairyFree LOGICAL, is_healthy LOGICAL, is_sustainable LOGICAL, is_lowFodmap LOGICAL, is_Popular LOGICAL, license VARCHAR(20), source_url VARCHAR(100));

In [20]:
logger.debug(dfrecipes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   is_vegetarian      100 non-null    bool   
 1   is_vegan           100 non-null    bool   
 2   is_glutenFree      100 non-null    bool   
 3   is_dairyFree       100 non-null    bool   
 4   is_healthy         100 non-null    bool   
 5   is_cheap           100 non-null    bool   
 6   is_Popular         100 non-null    bool   
 7   is_sustainable     100 non-null    bool   
 8   is_lowFodmap       100 non-null    bool   
 9   price_per_serving  100 non-null    float64
 10  recipe_title       100 non-null    object 
 11  ready_min          100 non-null    int64  
 12  servings           100 non-null    int64  
 13  source_url         100 non-null    object 
 14  summary            100 non-null    object 
 15  license            61 non-null     object 
 16  id_recipe          100 non-

### Ingredients table
(id_ingredient INT, ing_name VARCHAR(50), consistency VARCHAR(20), aisle VARCHAR(20));

In [21]:
logger.debug(dfIng.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   consistency    404 non-null    object
 1   ing_name       404 non-null    object
 2   aisle          403 non-null    object
 3   id_ingredient  404 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 12.8+ KB


### reference_ing table
(#id_recipe, #id_ingredient, measure VARCHAR(50));

In [22]:
logger.debug(dfreference_ing.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1053 entries, 0 to 1052
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   measure        1053 non-null   object
 1   id_recipe      1053 non-null   Int64 
 2   id_ingredient  1053 non-null   Int64 
dtypes: Int64(2), object(1)
memory usage: 26.9+ KB


### Equipment table
(id_equipment INT, equip_name VARCHAR(50));

In [23]:
logger.debug(dfequip.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   equip_name    48 non-null     object
 1   id_equipment  48 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 896.0+ bytes


### Instructions table
Instruction = (id_instruction INT, #id_recipe);

In [24]:
logger.debug(dfIns.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   instruction_id  97 non-null     int64
 1   id_recipe       97 non-null     Int64
dtypes: Int64(1), int64(1)
memory usage: 1.7 KB


### steps table
(id_step INT, step VARCHAR(8000), number INT, length VARCHAR(50), #id_instruction);

In [25]:
logger.debug(dfstep_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626 entries, 0 to 625
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   step            626 non-null    object 
 1   length          154 non-null    object 
 2   number          626 non-null    float64
 3   id_step         626 non-null    int64  
 4   instruction_id  626 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 24.6+ KB


### reference_equip table
(#id_recipe, #id_step, #id_equipment);

In [26]:
logger.debug(dfreference_equip.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755 entries, 0 to 754
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   id_recipe     755 non-null    Int64
 1   id_step       755 non-null    int64
 2   id_equipment  420 non-null    Int64
dtypes: Int64(2), int64(1)
memory usage: 19.3 KB


### dish_type table
(id_dish_type INT, dish_type VARCHAR(50))

In [27]:
logger.debug(dfdish_type.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   dish_type     25 non-null     object
 1   id_dish_type  25 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 528.0+ bytes


### is_a  table
(#id_recipe, #id_dish_type)

In [28]:
logger.debug(dfis_a.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355 entries, 0 to 354
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   id_dish_type  352 non-null    Int64
 1   id_recipe     355 non-null    Int64
dtypes: Int64(2)
memory usage: 6.4 KB


### cuisine table
(id_cuisine INT, recipe_cuisine VARCHAR(50));

In [29]:
logger.debug(dfcuisine.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   recipe_cuisine  18 non-null     object
 1   id_cuisine      18 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 416.0+ bytes


### belongs table
(#id_recipe, #id_cuisine);

In [30]:
logger.debug(dfbelongs.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   id_cuisine  57 non-null     Int64
 1   id_recipe   126 non-null    Int64
dtypes: Int64(2)
memory usage: 2.3 KB


# load

In [31]:
# Database configuration
DB_HOST='10.0.2.15' # Database host IP address
DB_PORT= 5432 # Database port number
DB_NAME='recipe_etl' # Name of the database
DB_USER='maryem' # Database user name
DB_PASSWORD='HelloWorld' # Database user password

try:
    # Establish connection to the database
    conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
    logger.info(f"Connected to {DB_NAME}")
except psycopg2.Error as e:
    logger.error(f"Error: Could not make connection to the database {DB_NAME}")
    logger.error(e)
    conn = None

2024-07-09 00:01:25,104 - __main__ - INFO - Connected to recipe_etl


In [32]:
# Function to check if a record already exists in the database
def record_exists(query, values):
    """
    Check if a record already exists in the database.

    Parameters:
    query (str): The SQL query to execute.
    values (tuple): The values to use in the SQL query.

    Returns:
    bool: True if the record exists, False otherwise.
    """
    try:
        # Execute the query with the provided values
        cur.execute(query, values)
        # Check if any record was fetched (exists)
        return cur.fetchone() is not None
    except psycopg2.Error as e:
        # Log an error if any occurs during the check
        logger.error(f"Error in checking for existing record: {e}")
        return False

# Function to execute an insert query and check if the insertion was successful
def execute_insert_and_check(query, values, table_name, check_query, check_values):
    """
    Execute an insert query and check if the insertion was successful.

    Parameters:
    query (str): The SQL insert query to execute.
    values (tuple): The values to use in the SQL insert query.
    table_name (str): The name of the table to insert into.
    check_query (str): The SQL query to check if the record already exists.
    check_values (tuple): The values to use in the check query.

    Raises:
    ValueError: If the table is empty after insertion attempts.
    """
    try:
        # Check if the record already exists
        if not record_exists(check_query, check_values):
            # Execute the insert query with the provided values
            cur.execute(query, values)
            # Commit the transaction
            conn.commit()
            # Check the number of rows in the table after the insert
            cur.execute(f"SELECT COUNT(*) FROM {table_name}")
            row_count = cur.fetchone()[0]
            # Raise an error if the table is empty after insertion
            if row_count == 0:
                logger.error(f"The {table_name} table is empty after insertion attempts.")
                raise ValueError(f"The {table_name} table is empty after insertion attempts.")
            else:
                # Log success if insertion was successful
                logger.info(f"Values inserted into {table_name} successfully")
        else:
            # Log a warning if a duplicate entry is found
            logger.warning(f"Duplicate entry found in {table_name}. Skipping insertion.")
    except psycopg2.Error as e:
        # Log an error if any occurs during the insertion
        logger.error(f"Error in inserting values into {table_name}: {e}", exc_info=True)
        # Revert the database to the state it was in before the current transaction began.
        conn.rollback()
        raise
    except ValueError as ve:
        # Log any value errors
        logger.error(ve)
        raise


In [33]:
# Proceed only if the connection was successful
if conn:
    try:
        # Obtain a cursor to execute queries
        cur = conn.cursor()
        # Log that the cursor was obtained successfully
        logger.info(f"Cursor obtained for the database {DB_NAME}")
    except psycopg2.Error as e:
        # Log an error if the cursor could not be obtained
        logger.error(f"Error: Could not get cursor for the database {DB_NAME}")
        logger.error(e)
        cur = None
    
    # If cursor is obtained successfully
    if cur:
        # Insert into Recipe table
        try:
            id_recipe_set=set()
            # Iterate over each row in the dataframe dfrecipes
            for index, row in dfrecipes.iterrows():
                # Check if the recipe already exists by id_recipe or recipe_title
                check_query = """
                SELECT 1 FROM Recipe WHERE recipe_title = %s
                """
                check_values = (row['recipe_title'],)

                # Execute the insert and check function for the Recipe table
                execute_insert_and_check(
                    """
                    INSERT INTO Recipe (id_recipe, recipe_title, ready_min, summary, servings, is_cheap, price_per_serving, 
                    is_vegetarian, is_vegan, is_glutenFree, is_dairyFree, is_healthy, is_sustainable, is_lowFodmap, 
                    is_Popular, license, source_url)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""",
                    (
                        int(row['id_recipe']),
                        row['recipe_title'],
                        int(row['ready_min']),
                        row['summary'],
                        int(row['servings']),
                        bool(row['is_cheap']),
                        float(row['price_per_serving']),
                        bool(row['is_vegetarian']),
                        bool(row['is_vegan']),
                        bool(row['is_glutenFree']),
                        bool(row['is_dairyFree']),
                        bool(row['is_healthy']),
                        bool(row['is_sustainable']),
                        bool(row['is_lowFodmap']),
                        bool(row['is_Popular']),
                        row['license'],
                        row['source_url'],
                    ),
                    'Recipe',
                    check_query,
                    check_values
                )
    
                # Check if the recipe was inserted
                cur.execute(check_query, check_values)
                if not cur.fetchone():
                    id_recipe_set.add(int(row['id_recipe'])) # Save the recipe ID for related table insertions
                    continue  # Skip to the next recipe if this one already exists
           
            # Insert into Instruction table
            try:
                id_ins_set=set()
                for index, row in dfIns.iterrows():
                    if int(row['id_recipe']) not in id_recipe_set:
                        execute_insert_and_check(
                            """
                            INSERT INTO Instruction (id_instruction, id_recipe)
                            VALUES (%s, %s);""",
                            (
                                int(row['instruction_id']),
                                int(row['id_recipe'])
                            ),
                            'Instruction',
                            "SELECT 1 FROM Instruction WHERE id_instruction = %s AND id_recipe = %s",
                            (int(row['instruction_id']), int(row['id_recipe']))
                        )
                    # Check if the instruction was inserted
                    cur.execute(check_query, check_values)
                    if not cur.fetchone():
                        id_ins_set.add(int(row['instruction_id'])) # Save the instruction ID for related table insertions
                        continue  # Skip to the next instruction if this one already exists
            except Exception as e:
                logger.error(f"Failed to insert into Instruction table: {e}", exc_info=True)
            
            # Insert into Ingredient table
            try:
                id_ing_set = set()
                for index, row in dfIng.iterrows():
                    # Check if the ingredient already exists by ing_name
                    check_query = """
                    SELECT 1 FROM Ingredient WHERE ing_name = %s
                    """
                    check_values = (row['ing_name'],)

                    execute_insert_and_check(
                        """
                        INSERT INTO Ingredient (id_ingredient, ing_name, consistency, aisle)
                        VALUES (%s, %s, %s, %s);""",
                        (
                            int(row['id_ingredient']),
                            row['ing_name'],
                            row['consistency'],
                            row['aisle']
                        ),
                        'Ingredient',
                        check_query,
                        check_values
                    )

                    # Check if the ingredient was inserted
                    cur.execute(check_query, check_values)
                    if cur.fetchone():
                        continue  # Skip to the next ingredient if this one already exists
                    cur.execute("""SELECT id_ingredient FROM Ingredient WHERE ing_name = %s""", (row['ing_name'],))
                    # Fetch the result
                    result = cur.fetchone()
                    # Extract the id_ingredient from the result
                    old_id = result[0]
                    # Append to the set
                    id_ing_set.add({int(row['id_ingredient']): old_id})

                # Initialize an empty dictionary to hold the final combined dictionary
                ing_id_MAP = {}
                # Iterate over the set and update the final dictionary with each dictionary
                for d in id_ing_set:
                    ing_id_MAP.update(d)
            except Exception as e:
                logger.error(f"Failed to insert into Ingredient table: {e}", exc_info=True)

            # Insert into Step table
            try:
                id_step_set=set()
                for index, row in dfstep_final.iterrows():
                    if int(row['instruction_id']) not in id_ins_set:
                        execute_insert_and_check(
                            """
                            INSERT INTO Step (id_step, step, number, length, id_instruction)
                            VALUES (%s, %s, %s, %s, %s);""",
                            (
                                int(row['id_step']),
                                row['step'],
                                int(row['number']),
                                row['length'],
                                int(row['instruction_id'])
                            ),
                            'Step',
                            "SELECT 1 FROM Step WHERE id_step = %s",
                            (int(row['id_step']),)
                    )
                    # Check if the step was inserted
                    cur.execute(check_query, check_values)
                    if not cur.fetchone():
                        id_recipe_set.add(int(row['id_step'])) # Save the step ID for related table insertions
                        continue  # Skip to the next step if this one already exists
            except Exception as e:
                logger.error(f"Failed to insert into Step table: {e}", exc_info=True)

            # Insert into Equipment table
            try:
                for index, row in dfequip.iterrows():
                    execute_insert_and_check(
                        """
                        INSERT INTO Equipment (id_equipment, equip_name)
                        VALUES (%s, %s);""",
                        (
                            int(row['id_equipment']),
                            row['equip_name']
                        ),
                        'Equipment',
                        "SELECT 1 FROM Equipment WHERE id_equipment = %s",
                        (int(row['id_equipment']),)
                    )
            except Exception as e:
                logger.error(f"Failed to insert into Equipment table: {e}", exc_info=True)

            # Insert into Dish table
            try:
                for index, row in dfdish_type.iterrows():
                    execute_insert_and_check(
                        """
                        INSERT INTO Dish (id_dish_type, dish_type)
                        VALUES (%s, %s);""",
                        (
                            int(row['id_dish_type']),
                            row['dish_type']
                        ),
                        'Dish',
                        "SELECT 1 FROM Dish WHERE id_dish_type = %s",
                        (int(row['id_dish_type']),)
                    )
            except Exception as e:
                logger.error(f"Failed to insert into Dish table: {e}", exc_info=True)

            # Insert into Cuisine table
            try:
                for index, row in dfcuisine.iterrows():
                    execute_insert_and_check(
                        """
                        INSERT INTO Cuisine (id_cuisine, recipe_cuisine)
                        VALUES (%s, %s);""",
                        (
                            int(row['id_cuisine']),
                            row['recipe_cuisine']
                        ),
                        'Cuisine',
                        "SELECT 1 FROM Cuisine WHERE id_cuisine = %s",
                        (int(row['id_cuisine']),)
                    )
            except Exception as e:
                logger.error(f"Failed to insert into Cuisine table: {e}", exc_info=True)

            # Insert into reference_ing table
            try:
                dfreference_ing = dfreference_ing.replace({pd.NA: np.nan})
                for index, row in dfreference_ing.iterrows():
                    if int(row['id_recipe']) not in id_recipe_set:
                        ing_id= int(row['id_ingredient'])
                        if ing_id in ing_id_MAP.keys():
                            ing_id=ing_id_MAP.get(ing_id)
                            logger.warning("altered ingredient id because it exists")
                        execute_insert_and_check(
                            """
                            INSERT INTO reference_ing (id_recipe, id_ingredient, measure)
                            VALUES (%s, %s, %s);""",
                            (
                                int(row['id_recipe']),
                                ing_id,
                                row['measure']
                            ),
                            'reference_ing',
                            "SELECT 1 FROM reference_ing WHERE id_recipe = %s AND id_ingredient = %s",
                            (int(row['id_recipe']), ing_id)
                        )
            except Exception as e:
                logger.error(f"Failed to insert into reference_ing table: {e}", exc_info=True)

            # Insert into reference_equip table
            try:
                dfreference_equip = dfreference_equip.replace({pd.NA: np.nan})
                for index, row in dfreference_equip.iterrows():
                    if int(row['id_recipe']) not in id_recipe_set:
                        execute_insert_and_check(
                            """
                            INSERT INTO reference_equip (id_recipe, id_step, id_equipment)
                            VALUES (%s, %s, %s);""",
                            (
                                int(row['id_recipe']),
                                int(row['id_step']) if pd.notna(row['id_step']) else None,
                                int(row['id_equipment']) if pd.notna(row['id_equipment']) else None
                            ),
                            'reference_equip',
                            "SELECT 1 FROM reference_equip WHERE id_recipe = %s AND id_step = %s AND id_equipment = %s",
                            (
                                int(row['id_recipe']),
                                int(row['id_step']) if pd.notna(row['id_step']) else None,
                                int(row['id_equipment']) if pd.notna(row['id_equipment']) else None
                            )
                            )
            except Exception as e:
                logger.error(f"Failed to insert into reference_equip table: {e}", exc_info=True)

            # Insert into belongs table
            try:
                dfbelongs = dfbelongs.replace({pd.NA: np.nan})
                for index, row in dfbelongs.iterrows():
                    if int(row['id_recipe']) not in id_recipe_set:
                        execute_insert_and_check(
                            """
                            INSERT INTO belongs (id_recipe, id_cuisine)
                            VALUES (%s, %s);""",
                            (
                                int(row['id_recipe']),
                                int(row['id_cuisine']) if pd.notna(row['id_cuisine']) else None
                            ),
                            'belongs',
                            "SELECT 1 FROM belongs WHERE id_recipe = %s AND id_cuisine = %s",
                            (
                                int(row['id_recipe']),
                                int(row['id_cuisine']) if pd.notna(row['id_cuisine']) else None
                            )
                            )
            except Exception as e:
                logger.error(f"Failed to insert into belongs table: {e}", exc_info=True)

            # Insert into is_a table
            try:
                # Replace pd.NA with np.nan
                dfis_a = dfis_a.replace({pd.NA: np.nan})
                for index, row in dfis_a.iterrows():
                    if int(row['id_recipe']) not in id_recipe_set:
                        execute_insert_and_check(
                            """
                            INSERT INTO is_a (id_recipe, id_dish_type)
                            VALUES (%s, %s);""",
                            (
                                int(row['id_recipe']),
                                int(row['id_dish_type']) if pd.notna(row['id_dish_type']) else None
                            ),
                            'is_a',
                            "SELECT 1 FROM is_a WHERE id_recipe = %s AND id_dish_type = %s",
                            (
                                int(row['id_recipe']),
                                int(row['id_dish_type']) if pd.notna(row['id_dish_type']) else None
                            )
                            )
            except Exception as e:
                logger.error(f"Failed to insert into is_a table: {e}", exc_info=True)

        finally:
            cur.close()
    else:
        logger.error("Cursor could not be obtained. Exiting the program.")
    conn.close()
else:
    logger.error("Connection to the database could not be established. Exiting the program.")


2024-07-09 00:01:25,165 - __main__ - INFO - Cursor obtained for the database recipe_etl
2024-07-09 00:01:25,176 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-09 00:01:25,181 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-09 00:01:25,186 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-09 00:01:25,191 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-09 00:01:25,228 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-09 00:01:25,233 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-09 00:01:25,238 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-09 00:01:25,243 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-09 00:01:25,248 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-09 00:01:25,255 - __main__ - INFO - Values inserted into Recipe successfully
2024-07-09 00:01:25,261 - __main__ - INFO - Values inserted in